# i-Learning 教材影片轉錄逐字稿

利用 OpenAI-Whisper 轉錄影片，請尊重智慧財產權！

In [27]:
#@title 👇 安裝環境 { display-mode: "form" }
#@markdown ---
#@markdown 建議將執行環境切換為 **GPU** :<br/>
#@markdown *執行環境->變更執行階段類型->硬體加速器, 選擇 GPU*
#@markdown
#@markdown ---

!pip install -q requests beautifulsoup4 lxml pycryptodome faster-whisper ipywidgets

import os
import requests
import json
import hashlib
from Crypto.Cipher import DES
import base64
from bs4 import BeautifulSoup
import time
import re
import subprocess
from faster_whisper import WhisperModel
from IPython.display import display
import ipywidgets as widgets

In [ ]:
#@title 👇 啟用 { form-width: "20%", display-mode: "form" }


os.environ['KMP_DUPLICATE_LIB_OK']='True'
url = "https://i-learning.cycu.edu.tw/"

# MD5 Encrypt
def md5_encode(input_string) -> str:
    md5_hash = hashlib.md5()
    md5_hash.update(input_string.encode('utf-8'))
    return md5_hash.hexdigest()

# DES Encrypt ECB NoPadding
def des_encode(key:str, data) -> str:
    cipher = DES.new(key.encode('utf-8'), DES.MODE_ECB)
    encrypted_data = cipher.encrypt(data.encode('utf-8'))
    return str(base64.encodebytes(encrypted_data),encoding='utf-8').replace("\n","")

def fetch_login_key(session):
    while True:
        with session.get(url + "sys/door/re_gen_loginkey.php?xajax=reGenLoginKey", headers=headers) as response:
            res = response.text
            if "loginForm.login_key.value = \"" in res:
                return res.split("loginForm.login_key.value = \"")[1].split("\"")[0]

def login(session, id, pwd, loginKey) -> bool:
    with session.post(url + "login.php", headers=headers, data={
        "username": id,
        "pwd": pwd,
        "password": "*" * len(pwd),
        "login_key": loginKey,
        "encrypt_pwd": des_encode(md5_encode(pwd)[:4] + loginKey[:4], pwd + " " * (16 - len(pwd) % 16) if len(pwd) % 16 != 0 else pwd),
    }) as response:
        res = response.text
        if "lang=\"big5" in res:
            return False
    return True

def fetch_courses(session):
    with session.get(url + "learn/mooc_sysbar.php", headers=headers) as response:
        soup = BeautifulSoup(response.text, 'lxml')
        courses = {
            option["value"]: option.text
            for child in soup.select("optgroup[label=\"正式生、旁聽生\"]")
            for option in child.find_all("option")
        }
        return courses

def fetch_videos(session, course_id) -> dict:
    with session.get(url + f"xmlapi/index.php?action=my-course-path-info&cid={course_id}", headers=headers) as response:
        items = json.loads(response.text)
        hrefs = dict()
        if items['code'] == 0:
            def search_hrefs(data):
                if isinstance(data, dict):
                    for key, value in data.items():
                        if key == 'href' and value.endswith('.mp4'):
                            pattern = r'[<>:"/\\|?*\x00-\x1F\x7F]'
                            name = re.sub(pattern,'',str(data['text']))
                            hrefs[name] = str(value)
                        elif isinstance(value, (dict, list)):
                            search_hrefs(value)
                elif isinstance(data, list):
                    for item in data:
                        search_hrefs(item)
            search_hrefs(items['data']['path']['item'])
        return hrefs

def downloadVideo(session, filename, href) -> str:
    with session.get(href, headers=headers, stream=True) as response:
        if response.status_code != 200:
            return
        filename += ".mp4"
        save_path = f"videos/"
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        file_path = os.path.join(save_path, filename)
        if os.path.exists(file_path):
            return file_path
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    file.flush()
        return file_path

def transcribe(model, videoPath, name):
    audioFile = extractAudio(videoPath)
    segments, _ = model.transcribe(audioFile, language="zh")
    save_path = f"videos/"
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    file_path = os.path.join(save_path, name + "_transcription.txt")
    with open(file_path, 'w', encoding="utf-8") as txt:
        for segment in segments:
            txt.write("[%.2fs -> %.2fs] %s\n" % (segment.start, segment.end, segment.text))
    os.remove(audioFile)

def extractAudio(video_file, output_ext="mp3"):
    filename, _ = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15"}

def main():
    os.system("title CYCU-iLearning-Video-Transcription")
    print("<!!! 尊重版權/著作權 尊重版權/著作權 尊重版權/著作權 !!!>")

    id_widget = widgets.Text(
        description='學號：',
        placeholder='輸入您的學號'
    )
    pwd_widget = widgets.Password(
        description='密碼：',
        placeholder='輸入您的itouch密碼'
    )
    login_button = widgets.Button(
        description='登入',
        button_style='success',
        layout=widgets.Layout(margin="10px 0px 0px 90px")
    )
    output = widgets.Output()

    def on_login_button_clicked(b):
        id = id_widget.value
        pwd = pwd_widget.value

        session = requests.session()
        login_key = fetch_login_key(session)
        if not login(session, id, pwd, login_key):
            with output:
                print("登入失敗，請重新再試!")
            return

        device_widget = widgets.RadioButtons(
            options=['CPU', 'CUDA (GPU)'],
            description='選擇運算方式'
        )
        model_widget = widgets.RadioButtons(
            options=['small 能力弱', 'medium 能力中等', 'large-v3 能力強'],
            description='選擇模型類型'
        )
        start_button = widgets.Button(
            description='開始',
            button_style='success',
            layout=widgets.Layout(margin="0px 0px 0px 90px")
        )

        def on_start_button_clicked(b):
            device = "cpu" if device_widget.value == 'CPU' else "cuda"
            model_choice = {'small 能力弱': 'small', 'medium 能力中等': 'medium', 'large-v3 能力強': 'large-v3'}
            model_name = model_choice[model_widget.value]

            download_root = "model/"
            if not os.path.exists(download_root):
                os.makedirs(download_root)
            model = WhisperModel(model_name, device=device, download_root=download_root, compute_type="auto")

            course_dropdown = widgets.Dropdown(
                options=[],
                description='選擇課程',
                layout=widgets.Layout(margin="10px 0px 0px 0px")
            )
            video_dropdown = widgets.Dropdown(
                options=[],
                description='選擇影片',
                layout=widgets.Layout(margin="10px 0px 0px 0px")
            )

            fetch_button = widgets.Button(
                description='獲取影片列表',
                button_style='info',
                layout=widgets.Layout(margin="10px 0px 0px 90px")
            )

            def on_fetch_button_clicked(b):
                course_id = course_dropdown.value
                hrefs = fetch_videos(session, course_id)

                if len(hrefs) == 0:
                    with output2:
                        output2.append_stdout("沒有找到影片")
                    return

                download_button = widgets.Button(
                    description='下載並轉錄',
                    button_style='info',
                    layout=widgets.Layout(margin="10px 0px 0px 90px")
                )

                def on_download_button_clicked(b):
                    video_name = video_dropdown.label
                    href = video_dropdown.value

                    with output:
                        output.clear_output()
                        output.append_stdout("影片下載中...")
                    start = time.time()
                    file_path = downloadVideo(session, video_name, href)
                    with output:
                        output.append_stdout("下載完成! 耗時: %.2fs" % (time.time() - start))

                    start = time.time()
                    with output:
                        output.append_stdout("AI轉錄中...")
                    transcribe(model, file_path, video_name)

                    output2 = widgets.Output()
                    with output2:
                      print("轉錄完成! 耗時: %.2fs" % (time.time() - start))
                      display(course_dropdown, fetch_button)
                    with output:
                      output.clear_output()
                      display(output2)

                download_button.on_click(on_download_button_clicked)
                video_dropdown.options = [(name, href) for name, href in hrefs.items()]
                output2 = widgets.Output()
                with output2:
                    display(video_dropdown, download_button)
                with output:
                    output.clear_output()
                    display(output2)

            fetch_button.on_click(on_fetch_button_clicked)

            output2 = widgets.Output()
            with output2:
                display(course_dropdown, fetch_button)
            with output:
                output.clear_output()
                display(output2)

            courses = fetch_courses(session)
            course_dropdown.options = [(courses[key], key) for key in courses]

        start_button.on_click(on_start_button_clicked)
        output2 = widgets.Output()
        with output2:
            display(device_widget, model_widget, start_button)
        with output:
            output.clear_output()
            display(output2)

    login_button.on_click(on_login_button_clicked)

    with output:
        display(id_widget, pwd_widget, login_button)

    display(output)

main()